In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path

println("Initializing Spark context...")
val conf = new SparkConf().setAppName("Example App")
val spark: SparkSession = SparkSession.builder.config(conf).getOrCreate()
//if you look in Spark Master UI, a application will be running after this

Intitializing Scala interpreter ...

Spark Web UI available at http://SIddhartha:4042
SparkContext available as 'sc' (version = 3.0.3, master = local[*], app id = local-1653458730490)
SparkSession available as 'spark'


Initializing Spark context...


import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.FileSystem
import org.apache.hadoop.fs.Path
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@68f538f7
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1347febf


In [2]:
println("************")
println("Hello, world!")
val rdd = spark.sparkContext.parallelize(Array(1 to 10))
rdd.count()
println("************")

************
Hello, world!
************


rdd: org.apache.spark.rdd.RDD[scala.collection.immutable.Range.Inclusive] = ParallelCollectionRDD[0] at parallelize at <console>:34


In [3]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")

df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]


In [4]:
df.show()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: integer (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



Q1)  Show the total number of Students in the file

In [6]:
df.count()

res3: Long = 1000


In [23]:
df("gender");

res9: org.apache.spark.sql.Column = gender


Q2) total marks achieved by male and female students

In [31]:
val df_q2 = df.select("gender","marks")
val total = df_q2.groupBy("gender").sum()
total.show()


+------+----------+
|gender|sum(marks)|
+------+----------+
|Female|     29636|
|  Male|     30461|
+------+----------+



df_q2: org.apache.spark.sql.DataFrame = [gender: string, marks: int]
total: org.apache.spark.sql.DataFrame = [gender: string, sum(marks): bigint]


Q3) Show the total number of students that have passed or failed

In [40]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val passed = df.filter(df("marks") >=50).select("marks").count()
print("The total number of students passed are  ",passed)
print("\n"+"--"*30+"\n")
val failed = df.filter(df("marks") <50).select("marks").count()
print("The total number of students failed are ",failed)

(The total number of students passed are  ,644)
------------------------------------------------------------
(The total number of students failed are ,356)

df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
passed: Long = 644
failed: Long = 356


Q4) show the total number of students enrolled per course


In [42]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val course_enrolled = df.groupBy("course").count()

print("\n"+"--"*30+"\n")
print("The total number of students enrolled per course are ")
print(course_enrolled.show())


------------------------------------------------------------
The total number of students enrolled per course are +------+-----+
|course|count|
+------+-----+
|    PF|  166|
|    DB|  157|
|   MVC|  157|
|   DSA|  176|
| Cloud|  192|
|   OOP|  152|
+------+-----+

()

df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
course_enrolled: org.apache.spark.sql.DataFrame = [course: string, count: bigint]


Q5) show the total marks student achieved per course

In [43]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val df_q5 = df.select("course","marks")
val total = df_q5.groupBy("course").sum()
total.show()


+------+----------+
|course|sum(marks)|
+------+----------+
|    PF|      9933|
|    DB|      9270|
|   MVC|      9585|
|   DSA|     10950|
| Cloud|     11443|
|   OOP|      8916|
+------+----------+



df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
df_q5: org.apache.spark.sql.DataFrame = [course: string, marks: int]
total: org.apache.spark.sql.DataFrame = [course: string, sum(marks): bigint]


Q6) show the average marks student achieved per course

In [47]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val df_q6 = df.select("course","marks")
val total = df_q6.groupBy("course").mean()
total.show()

print("\n"+"--"*30+"\n")
/*
from pyspark.sql.functions import format_number
total.select(format_number(total[1], 2).alias('avg_marks')).show()

print("\n"+"--"*30+"\n")
*/

+------+------------------+
|course|        avg(marks)|
+------+------------------+
|    PF| 59.83734939759036|
|    DB|59.044585987261144|
|   MVC| 61.05095541401274|
|   DSA| 62.21590909090909|
| Cloud|59.598958333333336|
|   OOP|  58.6578947368421|
+------+------------------+


------------------------------------------------------------


df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
df_q6: org.apache.spark.sql.DataFrame = [course: string, marks: int]
total: org.apache.spark.sql.DataFrame = [course: string, avg(marks): double]


Q7) show the minimum and maximum marks student achieved per course

In [49]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val df_q7 = df.select("course","marks")
val min = df_q7.groupBy("course").min().show()
val max = df_q7.groupBy("course").max().show()

+------+----------+
|course|min(marks)|
+------+----------+
|    PF|        20|
|    DB|        20|
|   MVC|        22|
|   DSA|        20|
| Cloud|        20|
|   OOP|        20|
+------+----------+

+------+----------+
|course|max(marks)|
+------+----------+
|    PF|        99|
|    DB|        98|
|   MVC|        99|
|   DSA|        99|
| Cloud|        99|
|   OOP|        99|
+------+----------+



df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
df_q7: org.apache.spark.sql.DataFrame = [course: string, marks: int]
min: Unit = ()
max: Unit = ()


Q8) show the average age of male and female students

In [51]:
val df = spark.read.option("header", true).option("inferSchema", true).csv("StudentData.csv")
val df_q8 = df.select("gender","age")
val total = df_q8.groupBy("gender").mean()
total.show()

print("\n"+"--"*30+"\n")
/*
from pyspark.sql.functions import format_number
total.select(format_number(total[1], 2).alias('avg_age')).show()

print("--"*30)
*/

+------+------------------+
|gender|          avg(age)|
+------+------------------+
|Female|28.489021956087825|
|  Male| 28.52304609218437|
+------+------------------+


------------------------------------------------------------


df: org.apache.spark.sql.DataFrame = [age: int, gender: string ... 5 more fields]
df_q8: org.apache.spark.sql.DataFrame = [gender: string, age: int]
total: org.apache.spark.sql.DataFrame = [gender: string, avg(age): double]
